### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [80]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [81]:
# rename columns to more user-friendly titles
school_data_renamed = school_data_complete.rename(columns={'Student ID':'Student ID','student_name':'Student Name','gender':'Gender','school_name':'School Name','grade':'Grade','reading_score':'Reading Score','math_score':'Math Score','School ID':'School ID','type':'Type','size':'Total Students','budget':'Total School Budget'})
school_data_sorted = school_data_renamed.sort_values("School Name")

In [82]:
# find total # of schools
Total_Schools = school_data_renamed["School Name"].nunique()

In [83]:
# determine total number of students overall
Total_Students = school_data_renamed["Student Name"].count()

In [84]:
# determine total budget, all schools
school_budgets = school_data_renamed['Total School Budget'].unique()
school_budget_sum = school_budgets.sum()
school_budget_sum

# determine avg math score and avg reading score
avg_math_score = school_data_renamed['Math Score'].mean()
avg_reading_score = school_data_renamed['Reading Score'].mean()

In [85]:
# find percent passing math, reading and overall pass rate

math_pass_df = school_data_renamed.loc[school_data_renamed["Math Score"] > 69, :]

math_pass_count = math_pass_df['Math Score'].count()
math_pass_percent = (math_pass_count / Total_Students)*100

# avg_read_pass_perc
read_pass_df = school_data_renamed.loc[school_data_renamed['Reading Score'] > 69,:]
read_pass_count = read_pass_df['Reading Score'].count()
read_pass_percent = (read_pass_count / Total_Students)*100

# percent overall pass rate
overall_pass_rate = (math_pass_percent + read_pass_percent) / 2

In [86]:
# create summary table of relevant data
data = {'Total Schools':[Total_Schools],
        'Total Students':[Total_Students],
        'Total School Budget':[school_budget_sum],
        'Average Math Score':[avg_math_score],
        'Average Reading Score':[avg_reading_score],
        '% Passing Math':[math_pass_percent],
        '% Passing Reading':[read_pass_percent],
        '% Overall Passing Rate':[overall_pass_rate]
       }
Summary_df = pd.DataFrame(data)

Summary_df2 = pd.DataFrame(data)
Summary_df2['Total Students'] = Summary_df['Total Students'].map('{:,}'.format)
Summary_df2['Total School Budget'] = Summary_df['Total School Budget'].map('${:,}'.format)
print("District Summary")
Summary_df2


District Summary


,Total Schools,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [87]:
# get rid of columns not used in School Summary questions / section
school_data_thin = school_data_sorted.drop(columns=['Student ID','Student Name','Gender','Grade','School ID'])

In [88]:
# create series of schools by number of students
school_size = school_data_thin.groupby('School Name')['Math Score'].count()

In [89]:
# find number of students passing math per school and percent of students at each school who passed math
passing_math = school_data_thin.loc[school_data_thin["Math Score"] > 69]
math_gpd = passing_math.groupby('School Name')['Math Score'].count()
math_perc_pass = (math_gpd / school_size)*100

In [90]:
# find number of students passing reading at each school and the percent of students passing reading
passing_reading = school_data_thin.loc[school_data_thin["Reading Score"] > 69]
reading_gpd = passing_reading.groupby('School Name')['Reading Score'].count()
reading_perc_pass = (reading_gpd / school_size)*100

In [91]:
# find overall pass rate (math pass perc + reading pass perc) / 2
overall_pass = (math_perc_pass + reading_perc_pass)/2

In [92]:
# find average math score per school
avg_math = school_data_thin.groupby('School Name')['Math Score'].mean()

# find average reading score per school
avg_reading = school_data_thin.groupby('School Name')['Reading Score'].mean()

In [93]:
# rearrange columns to have schools first and make more sense organizationally
org_school_data = school_data_sorted[['School Name','Type','Reading Score','Math Score', 'Total Students', 'Total School Budget','Student ID', 'Student Name','Gender', 'Grade','School ID']]

In [94]:
# find Budget per student at each school
per_stu_budget = org_school_data.groupby('School Name')['Total School Budget'].mean() / org_school_data.groupby('School Name')['Total Students'].mean()

In [95]:
# isolate "Type" so can drop duplicates and create a series of schools w their type
type_df = school_data_thin.drop(columns=['Reading Score', 'Math Score','Total Students','Total School Budget'])

In [96]:
# reduce list to one of each school with type
school_type = type_df.drop_duplicates('School Name')
school_type2 = school_type.set_index('School Name')

In [97]:
# find total school budget per school
total_school_budget = org_school_data.groupby('School Name')['Total School Budget'].mean()

In [98]:
# create new dataframe for School Summary, add columns
Summary_Schools_df = pd.DataFrame(columns=[])
Summary_Schools_df['Total Students'] = school_size
Summary_Schools_df['Total School Budget'] = total_school_budget
Summary_Schools_df['Per Student Budget'] = per_stu_budget
Summary_Schools_df['Average Math Score'] = avg_math
Summary_Schools_df['Average Reading Score'] = avg_reading
Summary_Schools_df['% Passing Math'] = math_perc_pass
Summary_Schools_df['% Passing Reading'] = reading_perc_pass
Summary_Schools_df['Overall Passing Rate'] = overall_pass

Summary_Schools_df['Type'] = school_type2

# reorder columns to match question order
Summary_Schools_df = Summary_Schools_df[['Type','Total Students','Total School Budget','Per Student Budget','Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate']]

# create a diff dataframe to hold formatting
Summary_Schools_df2 = Summary_Schools_df[['Type','Total Students','Total School Budget','Per Student Budget','Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate']]

Summary_Schools_df2['Total Students'] = Summary_Schools_df['Total Students'].map('{:,}'.format)
Summary_Schools_df2['Total School Budget'] = Summary_Schools_df['Total School Budget'].map('${:,}'.format)
Summary_Schools_df2['Per Student Budget'] = Summary_Schools_df['Per Student Budget'].map('${:,.2f}'.format)

Summary_Schools_df2.head(15)

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


In [99]:
# add "Type" back as column and reorder to answer questions
#Summary_Schools_df2['Type'] = school_type2
#Summary_Schools_df2 = Summary_Schools_df2[['Type','Total Students','Total School Budget','Per Student Budget','Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate']]
#Summary_Schools_df2

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [100]:
# find top 5 schools by overall passing rate
top_schools = Summary_Schools_df.sort_values(
    ["Overall Passing Rate"], ascending=False)
top_schools['Total Students'] = top_schools['Total Students'].map('{:,}'.format)
top_schools['Total School Budget'] = top_schools['Total School Budget'].map('${:,.2f}'.format)
top_schools['Per Student Budget'] = top_schools['Per Student Budget'].map('${:,.2f}'.format)
top_schools.head(5)

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [101]:
# find bottom 5 worst performing schools by passing rate
worst_schools = Summary_Schools_df.sort_values(
    ["Overall Passing Rate"], ascending=True)
worst_schools['Total Students'] = worst_schools['Total Students'].map('{:,}'.format)
worst_schools['Total School Budget'] = worst_schools['Total School Budget'].map('${:,.2f}'.format)
worst_schools['Per Student Budget'] = worst_schools['Per Student Budget'].map('${:,.2f}'.format)
worst_schools.head(5)

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [102]:
# create serieses for each grade
grade9 = org_school_data.loc[org_school_data["Grade"] == "9th"]
grade10 = org_school_data.loc[org_school_data["Grade"] == "10th"]
grade11 = org_school_data.loc[org_school_data["Grade"] == "11th"]
grade12 = org_school_data.loc[org_school_data["Grade"] == "12th"]

In [103]:
# find average math scores for each school year series and create new series
grade9_math = grade9.groupby('School Name')['Math Score'].mean()
grade10_math = grade10.groupby('School Name')['Math Score'].mean()
grade11_math = grade11.groupby('School Name')['Math Score'].mean()
grade12_math = grade12.groupby('School Name')['Math Score'].mean()

In [104]:
# create table with average math scores per grade by school
math_grades_df = pd.DataFrame(columns=[])
math_grades_df['9th Grade'] = grade9_math
math_grades_df['10th Grade'] = grade10_math
math_grades_df['11th Grade'] = grade11_math
math_grades_df['12th Grade'] = grade12_math
print("Average Math Scores by Grade")
math_grades_df

Average Math Scores by Grade


,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [105]:
# find average reading scores for each school year series and create new series
grade9_reading = grade9.groupby('School Name')['Reading Score'].mean()
grade10_reading = grade10.groupby('School Name')['Reading Score'].mean()
grade11_reading = grade11.groupby('School Name')['Reading Score'].mean()
grade12_reading = grade12.groupby('School Name')['Reading Score'].mean()

In [106]:
# create table with average math scores per grade by school
reading_grades_df = pd.DataFrame(columns=[])
reading_grades_df['9th Grade'] = grade9_reading
reading_grades_df['10th Grade'] = grade10_reading
reading_grades_df['11th Grade'] = grade11_reading
reading_grades_df['12th Grade'] = grade12_reading
print("Average Reading Scores by Grade")
reading_grades_df

Average Reading Scores by Grade


,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [107]:
# create bins for Spending per student values
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [108]:
# add bins as Spending per Student column to dataframe
Summary_Schools_df["Per Student Budget Range"] = pd.cut(Summary_Schools_df["Per Student Budget"], spending_bins, labels=group_names)


In [109]:
# drop columns not relevant to question regarding School Spending question
spending_perf_thin = Summary_Schools_df.drop(columns=['Type', 'Total Students', 'Total School Budget', 'Per Student Budget'])

In [110]:
# show averages for each bin
spend_grps = spending_perf_thin.groupby('Per Student Budget Range').mean()
print("Scores by Spending per Student")
spend_grps

Scores by Spending per Student


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget Range,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [111]:
# create bins
size_bins = [0, 1000, 2000, 5000]
group_names2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [112]:
# add bins as School Size column 
Summary_Schools_df["School Size"] = pd.cut(Summary_Schools_df["Total Students"], size_bins, labels=group_names2)

# drop irrelevant columns
size_perf_thin = Summary_Schools_df.drop(columns=['Type', 'Total Students', 'Total School Budget', 'Per Student Budget'])

In [113]:
# display data by bins
size_grps = size_perf_thin.groupby('School Size').mean()
print("Scores by School Size")
size_grps

Scores by School Size


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [114]:
# drop columns not relevant to question
summ_thin = Summary_Schools_df.drop(columns=['Total Students', 'Total School Budget', 'Per Student Budget','Per Student Budget Range','School Size'])

In [115]:
# display data by Type of school
type_perf = summ_thin.groupby('Type').mean()
print("Scores by School Type")
type_perf

Scores by School Type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [116]:
# written description of at least 2 observable trends

print('Observable trends:')
print('1) Looking at average scores and percent passing we see that schools with 2,000 - 5,000 students perform signficantly less well than students at the smaller schools.')
print('2) It is also quite clear that children at charter schools perform better in both Math and Reading.  This difference is more pronounced in Math.  This trend is also reflected when we look at the top performing schools versus the bottom in terms of Overall Rating.  The top five performing schools are ALL charter schools.')      
print('3) Students at schools that spend the least amount of money per student perform better in both math and reading as compared to the schools who spend the most per student.  This is the case in terms of average scores and percent passing.  This is likely correlated with the size of school.')


Observable trends:
1) Looking at average scores and percent passing we see that schools with 2,000 - 5,000 students perform signficantly less well than students at the smaller schools.
2) It is also quite clear that children at charter schools perform better in both Math and Reading.  This difference is more pronounced in Math.  This trend is also reflected when we look at the top performing schools versus the bottom in terms of Overall Rating.  The top five performing schools are ALL charter schools.
3) Students at schools that spend the least amount of money per student perform better in both math and reading as compared to the schools who spend the most per student.  This is the case in terms of average scores and percent passing.  This is likely correlated with the size of school.
